In [1]:
# REQUIRED IMPORTS FROM STANDARD PACKAGES

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import random
import pandas as pd
from os.path import join as pjoin
from glob import glob
import sklearn as sk
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import RandomizedSearchCV
import scipy
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import make_scorer, accuracy_score
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.externals.joblib import parallel_backend
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile, f_classif, chi2
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
#news
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


# IMPORTS FROM THE UTIL LIBRARY PROVIDED BY US

import util.vis as V
import util.helpers as H

# Normally, all libraries are loaded only once, 
# even if you execute the import code multiple times
# This code is helpful if you make your own helper libraries 
%load_ext autoreload
%autoreload 1
# list your libraries below with aimport: should ensure 
#they are reloaded each time without having to restart your kernel
# in this case, our libraries are used as an example

%aimport util.helpers, util.vis
%aimport features_extraction
%aimport augmentation
%aimport Analysis
%aimport validation
%aimport preprocessing

# seed random generator such that this notebook always returns the same values 
# (this is by no means necessary, but it is useful for reproducability of results)
rng = np.random.RandomState(42)

In [2]:
# PATHS

DATA_DIR = '../data'
POSE_DIR = '../data/pose'

## Loading data
Loading all the training and test data

In [3]:
def keep_sample(sample):
    counter = 0
    no_hands_present = False
    for frame in sample:
        if (frame[4][0] == 0 or frame[7][0] == 0):
            counter += 1
        if (frame[4][0] == 0 and frame[7][0] == 0):
            no_hands_present = True
    if counter/len(sample) > 0.5 or no_hands_present:
        return False
    else:
        return True
##Loading all the training data
dataset_file=pjoin(DATA_DIR,'labels.csv')

train_samples=[]
train_labels=[]
train_persons = []
train_personlabels = []

with open(dataset_file) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    sample_index = 0
    preprocessing_outliers = [229, 297, 1316, 1780, 1817, 49, 495, 541, 746, 878, 2910]
    outliers_jens_fixing_gap1 = [85 , 398 , 717 , 766 , 923 , 979 ,1111 ,1280 ,1671 ,1778 ,1960 ,1993 ,2038 ,2089, 2097, 2135 ,2140, 2145, 2417 ,2463 ,2554 ,2579, 2594, 2794, 2832 ,2975, 3138, 3211, 3215, 3325 ,3341 ,3396 ,3399 ,3482 ,3566, 3649, 3702]
    outliers_jens_fixing_gap2 = [264,322,578,592,663,717,766,770,885,888,894,901,906,917,923,933,949,954,964,979,986,1117,1151,1152,1159,1161,1171,1199,1227,1257,1297,1573,1600,1607,1665,1671,1684,1701,1767,1778,1831,1859,1883,1930,1931,1932,1943,1960,1972,1993,1999,2037,2050,2056,2060,2089,2091,2097,2106,2119,2135,2140,2167,2191,2209,2231,2232,2235,2260,2268,2289,2334,2336,2538,2551,2554,2579,2602,2614,2637,2683,2688,2694,2699,2726,2741,2787,2803,2810,2841,2844,2845,2848,2887,2888,2893,2902,2943,2947,2954,2967,2975,2984,2990,3037,3064,3073,3078,3087,3103,3118,3119,3136,3138,3149,3170,3283,3293,3298,3305,3332,3342,3435,3475,3482,3561,3589,3590,3621,3629,3641,3649,3658,3672,3693,3697,3699,3702,3709,3710]
    outliers_robbe_fixing_gap = [2538, 1753, 1327, 2954, 2956, 2958, 4, 1111, 1117, 1118, 2167, 1992, 1993, 1573, 1575, 1576, 977, 245, 1846, 1909, 2435, 2545, 2590, 2815, 3515, 396, 421, 434, 494, 676, 718, 795, 815, 821, 882, 977, 991, 1016, 1035, 1111, 1295, 1308, 1524, 1564, 1586, 1633, 1634, 1804]
    #add [2538, 1753, 1327, 2954, 2956, 2958, 4, 1111, 1117, 1118]
    for row in reader: 
        name, _gloss, label, _person = row
        sample = np.load(pjoin(POSE_DIR, 'train', name+'.npy'))
        #if (sample_index not in preprocessing_outliers) and (sample_index not in outliers_jens_fixing_gap1) and (sample_index not in outliers_jens_fixing_gap2) and (sample_index not in outliers_robbe_fixing_gap):            
        if keep_sample(sample):
            train_samples.append(sample)
            train_labels.append(int(label))
            train_persons.append(_person)
            train_personlabels.append((label, _person))
        sample_index += 1

train_samples=np.array(train_samples)
train_labels=np.array(train_labels)

##Loading all the test data
all_test_files = sorted(glob(pjoin(POSE_DIR, 'test', '*.npy')))  

test_samples = []
for numpy_file in all_test_files:
    sample = np.load(numpy_file)
    test_samples.append(sample)
    
test_samples = np.array(test_samples)

## Data preprocessing

In [4]:
train_samples = preprocessing.centering(train_samples)
test_samples = preprocessing.centering(test_samples)

--- Centering finished ---
--- Centering finished ---


## Extracting features

In [5]:
#Train data
df = features_extraction.extract_features(train_samples)
X_train = df.values
y_train = np.array(train_labels)
#Test data
X_test = features_extraction.extract_features(test_samples).values
#Number of total features
num_features = X_train.shape[1]
print("Number of features : ", num_features)
#Correlation Matrix
#Analysis.plot_correlation_matrix(df)

D:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3250: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)
D:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
D:\Anaconda3\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
D:\Anaconda3\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
D:\Anaconda3\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\main\Documents\ML_competition\features_extraction.py:1243: RuntimeWarning: invalid value encountered in true_divide
  ve

Number of features :  869


## Loss functions
Custom scoring functions

In [6]:
##Custom scoring functions

def top3_acc_score(y_true, proba):
    return H.mapk(proba,y_true)

def map3_score(y_true, proba):
    return H.top3_accuracy(proba, y_true)
    

acc = make_scorer(accuracy_score)
top3_acc = make_scorer(top3_acc_score, needs_proba=True) 
map3 = make_scorer(map3_score, needs_proba=True)
scoring_functions = {"acc": acc, "top3_acc": top3_acc, "map3": map3}

## Validation strategy

In [7]:
##Train validate data splitter for Cross Validation
seed = np.random.randint(1,999)
sgkf = validation.stratified_group_k_fold(train_samples, train_labels, train_persons, 5, seed)

## Pipeline & model training

In [22]:
# the function below generates our rescaled pipeline model
# with optimized hyperparameters (e.g.: regularisation parameter)
def tune_pipeline(x_data,r_data,verbose=0):
    pipe = Pipeline([
        ('scale', StandardScaler()), 
        ('selectkbest', SelectKBest(f_classif)), 
        ('mlp',MLPClassifier())
    ])

    
    # Set the parameters by cross-validation
    tuned_parameters = {'mlp__activation': ['tanh'], #['relu', 'logistic', 'tanh'],
                        'mlp__solver': ['sgd'], #['sgd', 'adam'],
                        'mlp__learning_rate': ['constant', 'adaptive'] ,#['constant', 'adaptive'], 
                        'mlp__alpha': np.arange(0.001, 0.01, 0.003),
                        'mlp__validation_fraction': [0.1, 0.2],
                       'selectkbest__k': np.arange(250, 330, 20)}
        
    print("------ Start tuning hyperparameters ------")
    CV = GridSearchCV(pipe, tuned_parameters, n_jobs=-1, scoring=map3, pre_dispatch='n_jobs', cv=sgkf, verbose=2)
    print(CV)
    with parallel_backend('threading'):
        CV.fit(x_data, r_data)
    print("------ Tuning hyperparameters finished ------")
    
    bestActivation = CV.best_params_['mlp__activation']
    bestLearningRate = CV.best_params_['mlp__learning_rate']
    bestSolver = CV.best_params_['mlp__solver']
    bestK = CV.best_params_['selectkbest__k']
    bestAlpha = CV.best_params_['mlp__alpha']
    bestFraction = CV.best_params_['mlp__validation_fraction']
    print("Optimal activation: ", bestActivation)
    print("Optimal Solver: ", bestSolver)
    print("Optimal learning rate: ", bestLearningRate)
    print("Optimal k: ", bestK)
    print("Optimal Alpha: ",  bestAlpha)
    print("Optimal Fraction: ", bestFraction)
    optimal_pipe = Pipeline([
        ('scale', StandardScaler()), 
        ('selectkbest', SelectKBest(f_classif,k=bestK)),
        #('pca', PCA(n_components=bestK)),
        ('mlp',MLPClassifier(alpha = bestAlpha, validation_fraction=bestFraction ,activation=bestActivation, solver=bestSolver, learning_rate=bestLearningRate))])
    
    optimal_pipe.fit(x_data,r_data)  
    
    if verbose>0:
    
        print("Grid scores on training data set:")
        print()
        cv_means = CV.cv_results_['mean_test_score']
        print(cv_means)
        cv_stds = CV.cv_results_['std_test_score']
        for mean, std, params in zip(cv_means, cv_stds, CV.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

    return optimal_pipe, CV.cv_results_

In [23]:
sgkf = validation.stratified_group_k_fold(train_samples, train_labels, train_persons, 5, seed)
optimal_pipe, res = tune_pipeline(X_train, y_train, 1)
print('---------Optimal pipe finished -----------')
##Train validate data splitter for Cross Validation
sgkf = validation.stratified_group_k_fold(train_samples, train_labels, train_persons, 5, seed)

with parallel_backend('threading'):
    scores = cross_val_score(optimal_pipe, X_train, y_train, scoring=map3, cv=sgkf, n_jobs=-1, pre_dispatch='n_jobs')
print(scores)
print("Average (cross validated) map@3 score: ",scores.mean(),", stdev: ",scores.std())

optimal_pipe.fit(X_train, y_train)

------ Start tuning hyperparameters ------
GridSearchCV(cv=<generator object stratified_group_k_fold at 0x00000149A3F246D8>,
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('selectkbest', SelectKBest(k=10, score_func=<function f_classif at 0x00000149A8EAB1E0>)), ('mlp', MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsi...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'mlp__activation': ['tanh'], 'mlp__solver': ['sgd'], 'mlp__learning_rate': ['constant', 'adaptive'], 'mlp__alpha': array([0.001, 0.004, 0.007, 0.01 ]), 'mlp__validation_fraction': [0.1, 0.2], 'selectkbest__k': array([250, 270, 290, 310])},
       pre_dispatch='n_jobs', refit=True, return_train_score='warn',
       sco

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
D:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  mlp__activation=tanh, mlp__alpha=0.001, mlp__learning_rate=constant, mlp__solver=sgd, mlp__validation_fraction=0.1, selectkbest__k=270, total=  24.6s
[CV] mlp__activation=tanh, mlp__alpha=0.001, mlp__learning_rate=constant, mlp__solver=sgd, mlp__validation_fraction=0.1, selectkbest__k=270 


D:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  mlp__activation=tanh, mlp__alpha=0.001, mlp__learning_rate=constant, mlp__solver=sgd, mlp__validation_fraction=0.1, selectkbest__k=250, total=  25.1s
[CV] mlp__activation=tanh, mlp__alpha=0.001, mlp__learning_rate=constant, mlp__solver=sgd, mlp__validation_fraction=0.1, selectkbest__k=270 


...

Grid scores on training data set:

[0.79129213 0.79775281 0.81011236 0.80168539 0.8002809  0.80393258
 0.80421348 0.80589888 0.79719101 0.79803371 0.8011236  0.8002809
 0.79466292 0.80674157 0.80280899 0.8008427  0.79438202 0.80421348
 0.8005618  0.79691011 0.79775281 0.7997191  0.8011236  0.79325843
 0.7991573  0.80140449 0.80365169 0.7988764  0.79578652 0.80589888
 0.80168539 0.79747191 0.79747191 0.80421348 0.80337079 0.8
 0.79831461 0.80505618 0.79662921 0.79859551 0.79831461 0.8008427
 0.79494382 0.7988764  0.80308989 0.79859551 0.8008427  0.79606742
 0.80674157 0.80561798 0.8008427  0.80786517 0.7997191  0.80674157
 0.80589888 0.80168539 0.80365169 0.7994382  0.7991573  0.80280899
 0.80224719 0.79522472 0.80449438 0.80140449]
0.791 (+/-0.037) for {'mlp__activation': 'tanh', 'mlp__alpha': 0.001, 'mlp__learning_rate': 'constant', 'mlp__solver': 'sgd', 'mlp__validation_fraction': 0.1, 'selectkbest__k': 250}
0.798 (+/-0.042) for {'mlp__activation': 'tanh', 'mlp__alpha': 0.001, 'mlp__

D:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklea

[0.8125     0.76836158 0.81680441 0.82730924 0.79703704]
Average (cross validated) map@3 score:  0.8044024527238463 , stdev:  0.02048068963395105


D:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('selectkbest', SelectKBest(k=290, score_func=<function f_classif at 0x00000149A8EAB1E0>)), ('mlp', MLPClassifier(activation='tanh', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsi...e=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])

## Generate kaggle submission file

In [24]:
test_probas = optimal_pipe.predict_proba(X_test)
H.create_submission(test_probas, 'mlp_classifier1.csv')